# Retrieve Player IDs
* The players dataset is stored as 'players_ids.csv'

In [13]:
import pandas as pd
from nba_api.stats.endpoints import commonallplayers
from datetime import datetime
import csv
import pprint

In [12]:
# Dictionary to map TEAM_ID to TEAM_NAME
team_id_to_name = {
    '1610612737': 'Atlanta Hawks',
    '1610612738': 'Boston Celtics',
    '1610612739': 'Cleveland Cavaliers',
    '1610612740': 'New Orleans Pelicans',
    '1610612741': 'Chicago Bulls',
    '1610612742': 'Dallas Mavericks',
    '1610612743': 'Denver Nuggets',
    '1610612744': 'Golden State Warriors',
    '1610612745': 'Houston Rockets',
    '1610612746': 'LA Clippers',
    '1610612747': 'Los Angeles Lakers',
    '1610612748': 'Miami Heat',
    '1610612749': 'Milwaukee Bucks',
    '1610612750': 'Minnesota Timberwolves',
    '1610612751': 'Brooklyn Nets',
    '1610612752': 'New York Knicks',
    '1610612753': 'Orlando Magic',
    '1610612754': 'Indiana Pacers',
    '1610612755': 'Philadelphia 76ers',
    '1610612756': 'Phoenix Suns',
    '1610612757': 'Portland Trail Blazers',
    '1610612758': 'Sacramento Kings',
    '1610612759': 'San Antonio Spurs',
    '1610612760': 'Oklahoma City Thunder',
    '1610612761': 'Toronto Raptors',
    '1610612762': 'Utah Jazz',
    '1610612763': 'Memphis Grizzlies',
    '1610612764': 'Washington Wizards',
    '1610612765': 'Detroit Pistons',
    '1610612766': 'Charlotte Hornets'
}

def get_active_players(start_season, end_season, output_file):
    # Initialize an empty list to store DataFrames for each season
    all_seasons_data = []

    # Extract the start and end years from the season strings
    start_year = int(start_season[:4])
    end_year = int(end_season[:4])
    
    # Loop through seasons from start_season to end_season
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year+1)[-2:]}"
        
        print(f"Fetching active players for season {season}...")
        
        # Fetch active players for the season
        season_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, league_id='00', season=season).get_data_frames()[0]
        
        # Add a 'SEASON' column to keep track of the season for each player
        season_data['SEASON'] = season

        # Keep only the necessary columns: Name, Team ID, and Player ID
        season_data = season_data[['DISPLAY_FIRST_LAST', 'TEAM_ID', 'PERSON_ID']]

        # Replace 'TEAM_ID' with the corresponding team name using the dictionary
        season_data['TEAM_NAME'] = season_data['TEAM_ID'].astype(str).map(team_id_to_name)
        
        # Drop the 'TEAM_ID' column
        season_data = season_data.drop(columns=['TEAM_ID'])
        
        # Append the data to the list
        all_seasons_data.append(season_data)
    
    # Combine all seasons' data
    combined_data = pd.concat(all_seasons_data, ignore_index=True)
    
    # Save to CSV
    combined_data.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

get_active_players('2020-21', '2023-24', 'player_ids.csv')


Fetching active players for season 2020-21...
Fetching active players for season 2021-22...
Fetching active players for season 2022-23...
Fetching active players for season 2023-24...
Data saved to player_ids.csv
